### Programa que calcula los TOAs y residuos de las observaciones

Importamos los paquetes que vamos a usar

In [ ]:
import numpy as np
import glob                         # para hacer listas de archivo
import subprocess                   # para usar subprocesos

import os
import shutil

Cargamos los tobs

In [ ]:
datafile = "results_total.txt"                                                  # archivo que contiene los tobs y S/N

files = np.loadtxt( datafile , skiprows=0 , dtype=str , usecols=(0) )
sn_obs = np.loadtxt( datafile , skiprows=0 , dtype=float , usecols=(1) )
t_obs = np.loadtxt( datafile , skiprows=0 , dtype=float , usecols=(2) )

Parámetros usados

In [ ]:
tmin = (2500/60)/2                # tiempo mínimo que deben durar las observaciones (min)

tim_out = "J0437-4715.tim"        # nombre del archivo .tim de salida
template = "J0437-4715.std"       # template a usar
parfile = "J0437-4715_JUMPS.par"  # nombre del archivo .par

Calculamos TOAs

In [ ]:
dirName = './pfds/'

os.chdir(dirName)                                       # entramos al directorio que contiene los .pfds
pfd_files = glob.glob('*pfd')                           # lista de archivos .pfd

print("Número de observaciones : " + str((len(pfd_files))))

# primero, generamos un string que contenga todos los archivos .pfd

archivos = ''
for pfd in pfd_files:
    if  t_obs[ np.where(files == pfd)[0][0] ] > tmin # pedimos que las observaciones duren al menos t_min  
        
        archivos = archivos + pfd + ' '
        
# ahora llamamos a pat para generar el archivo .tim
    
subprocess.call("pat -FT -A PGS -s " + template + " -f >" + tim_out + " 'tempo2' " + archivos, shell = True)
    
os.chdir('..')                                 # salimos del directorio que contiene los .pfds
    
shutil.copy(dirName + tim_out, "./")           # copiamos el archivo .tim a la carpeta

Calculamos residuos

In [ ]:
resfile = "./residues_.dat"   # nombre del archivo de residuos
        
subprocess.check_call( "tempo2 >" + resfile + " -us -f " + parfile + " " + tim_out, shell = True)       # calculamos los residuos
    
rms = np.genfromtxt ( resfile, comments="none", dtype=float, skip_header=21, max_rows=1, usecols=(10) ) # buscamos el RMS

print("RMS = " + str(rms))